In [119]:
import pandas as pd
import numpy as np
from assign_unique_ids_functions import keep_duplicates, remove_duplicates, aggregate_data

In [120]:
ase = pd.read_csv('../input/ase-units.csv.gz')
asu = pd.read_csv('../input/all-sworn-units.csv.gz')

In [121]:
df_list = [ase.copy(), asu.copy()]
uid_col = 'UID'
unit_col = 'Unit'
start_col = 'Start.Date'
end_col = 'End.Date'
resignation_col='Resignation.Date'

In [122]:
uh_df = pd.DataFrame()

In [123]:
for df in df_list:
    df = df.loc[:,[uid_col, unit_col, start_col, end_col]]
    df.dropna(subset = [unit_col, uid_col, start_col], how='any', inplace=True)
    df.drop_duplicates(inplace=True)
    print('{} rows with non-NA end date and end date before/equal to start date'.format(df[(df[end_col].notnull()) & 
                                                                                  (df[end_col] <= df[start_col])].shape[0]))
    
    df.loc[(df[end_col].notnull()) & (df[end_col] <= df[start_col]), end_col] = np.nan
    uh_df = uh_df.append(df)

6933 rows with non-NA end date and end date before/equal to start date
6937 rows with non-NA end date and end date before/equal to start date


In [124]:
uh_df.drop_duplicates(inplace=True)

In [125]:
uh_df.head()

UID  Unit  Start.Date    End.Date
0    1  14.0  2007-03-29         NaN
2    2  15.0  2007-03-29         NaN
4    3   1.0  1976-08-19  1976-08-20
5    3   4.0  1971-01-16  1976-08-18
6    3  22.0  1970-06-15  1971-01-15

In [126]:
prof = pd.read_csv('../input/officer-profiles.csv.gz')[[uid_col, resignation_col]]
prof = prof[(prof[uid_col].isin(uh_df[uid_col])) & (prof[resignation_col].notnull())]

In [127]:
uh_df = uh_df.merge(prof, on=uid_col, how='left')

In [128]:
indexes = ((uh_df[resignation_col].notnull()) &
           (uh_df[end_col].isnull()) &
           (uh_df[start_col] < uh_df[resignation_col]))

In [129]:
uh_df.loc[indexes, end_col] =  uh_df.loc[indexes, resignation_col]

In [130]:
uh_df.head()

UID  Unit  Start.Date    End.Date Resignation.Date
0    1  14.0  2007-03-29         NaN              NaN
1    2  15.0  2007-03-29         NaN              NaN
2    3   1.0  1976-08-19  1976-08-20              NaN
3    3   4.0  1971-01-16  1976-08-18              NaN
4    3  22.0  1970-06-15  1971-01-15              NaN

In [131]:
uh_df.drop(resignation_col, axis=1, inplace=True)

In [132]:
uh_df.head()

UID  Unit  Start.Date    End.Date
0    1  14.0  2007-03-29         NaN
1    2  15.0  2007-03-29         NaN
2    3   1.0  1976-08-19  1976-08-20
3    3   4.0  1971-01-16  1976-08-18
4    3  22.0  1970-06-15  1971-01-15

In [133]:
uh_rd = remove_duplicates(uh_df,['UID','Start.Date', 'Unit'])
uh_kd = aggregate_data(keep_duplicates(uh_df, [uid_col, start_col, unit_col]),
                       uid=uid_col, id_cols=[start_col, unit_col], max_cols = [end_col])

In [134]:
assert uh_rd.shape[0] + uh_kd.shape[0] == uh_df[[uid_col, unit_col, start_col]].drop_duplicates().shape[0],\
        'UH.. missing rows!'

In [135]:
uh_df = uh_rd.append(uh_kd)
uh_df.sort_values([uid_col, start_col, unit_col], inplace=True)
uh_df.reset_index(drop=True, inplace=True)

In [136]:
uh_df.head()

End.Date  Start.Date  UID  Unit
0  2016-06-04  2007-03-29    1  14.0
1         NaN  2016-06-05    1   3.0
2         NaN  2007-03-29    2  15.0
3  1971-01-15  1970-06-15    3  22.0
4  1976-08-18  1971-01-16    3   4.0